## HOMEWORK (Winter 7.12.2)
Let’s assess the degree to which perceptual words differ in terms of iconicity as a function of sensory modality, as explored in Winter et al. (2017) (see Chapter 2). 
The following code loads in the Lynott and Connell (2009) modality ratings for adjectives and our iconicity ratings. 
The two tibbles are then merged, and a subset of the columns is extracted using `select()`.

In [1]:

# nifty code using the pacman package
# it checks if the packages specified below are installed, if not, they will be installed, if yes, they will be loaded
if (!require("pacman")) install.packages("pacman")
pacman::p_load(rstudioapi, tidyverse, broom)

# set the current working directory to the one where this file is
# current_working_dir <- dirname(rstudioapi::getActiveDocumentContext()$path)
# setwd(current_working_dir)


Loading required package: pacman



In [2]:
# load in data
lyn <- read_csv("lynott_connell_2009_modality.csv")
icon <- read_csv('perry_winter_2017_iconicity.csv')

# Merge data
both <- 
  left_join(lyn, icon) %>% 
  select(Word, DominantModality, Iconicity)

head(both)

Rows: 423 Columns: 9

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): PropertyBritish, Word, DominantModality
dbl (6): Sight, Touch, Sound, Taste, Smell, ModalityExclusivity


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Rows: 3001 Columns: 8

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Word, POS
dbl (6): SER, CorteseImag, Conc, Syst, Freq, Iconicity


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Joining, by = "Word"



Word,DominantModality,Iconicity
<chr>,<chr>,<dbl>
abrasive,Haptic,1.3125000
absorbent,Visual,0.9230769
aching,Haptic,0.2500000
acidic,Gustatory,1.0000000
acrid,Olfactory,0.6153846
adhesive,Haptic,1.3333333


In [3]:
sum(is.na(both))

[1] 1

In [4]:
both = drop_na(both)

Fit a linear model where `Iconicity` is modeled as a function of the categorical predictor `DominantModality`. Write a little summary of what the output of the model means.
Can you use the coefficients to derive predictions for all five categories?
Compare your results against the descriptive means, for which you can use `group_by()` and `summarize()`.


In [5]:
lm(Iconicity ~ DominantModality, data=both, singular.ok=FALSE)


Call:
lm(formula = Iconicity ~ DominantModality, data = both, singular.ok = FALSE)

Coefficients:
              (Intercept)  DominantModalityGustatory  
                   2.2561                    -1.4506  
   DominantModalityHaptic  DominantModalityOlfactory  
                  -0.4436                    -1.2445  
   DominantModalityVisual  
                  -1.0330  


The intercept resembles Auditory modalities with respect to iconicity, while the 
other coefficients resemble the differences between each other

In [6]:
model = lm(Iconicity ~ DominantModality, data=both, singular.ok=FALSE)
model_summary = summary(model)
coefs = coef(model_summary)
coefs

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),2.2561066,0.1249354,18.058190,2.846922e-54
DominantModalityGustatory,-1.4505511,0.1877883,-7.724397,8.443882e-14
DominantModalityHaptic,-0.4435719,0.1754187,-2.528646,1.181826e-02
DominantModalityOlfactory,-1.2445142,0.2375542,-5.238865,2.569915e-07
DominantModalityVisual,-1.0329595,0.1442629,-7.160255,3.663521e-12


In [7]:
intercept = coefs[1, 1]

From the model parameters, we can see which categorical variable that has the most and least iconicity. Essentially, the prediction is the dot product between the input vector and the coefficient summed with the intercept. We can consider the categorical variable as a one-hot encoded vector where there is one entry for each category:  

A vector representing the Gustatory category can thus be represented as the following: [1 0 0 0]

An Visual would then be [0 0 0 1]

Example:

Iconicity for visual gets predicted as 

[0 0 0 1] * [-1.45 -0.44 -1.25 -1.03] + 2.256 = 
-1.03 + 2.256 = 1.226

Since the word only can have a single category, the formula simply becomes the coefficient for the category plus the intercept.

In [8]:
as.matrix(coefs[2:5,0:1] + intercept)

DominantModalityGustatory,0.8055556
DominantModalityHaptic,1.8125347
DominantModalityOlfactory,1.0115925
DominantModalityVisual,1.2231472


We can ensure that these are the correct predictions by using the predict function and comparing

In [13]:
x = tibble(DominantModality=sort(unique(both$DominantModality)))
x$PredictedIconicity = predict(model, x)
x

DominantModality,PredictedIconicity
<chr>,<dbl>
Auditory,2.2561066
Gustatory,0.8055556
Haptic,1.8125347
Olfactory,1.0115925
Visual,1.2231472


In [14]:
means = both %>% group_by(DominantModality) %>% summarize(mean(Iconicity))
x$Means = means[["mean(Iconicity)"]]
x

DominantModality,PredictedIconicity,Means
<chr>,<dbl>,<dbl>
Auditory,2.2561066,2.2561066
Gustatory,0.8055556,0.8055556
Haptic,1.8125347,1.8125347
Olfactory,1.0115925,1.0115925
Visual,1.2231472,1.2231472


Turns out the results are identical

In [12]:
# This part is for one-hot encoding the categorical variables, but I decided to not include it 

# for (i in categories) {
#     new_df[[i]] = as.factor(dominant_modality == i)
# }

# model <- lm(both$Iconicity ~ (Haptic + Visual + Auditory + Gustatory + Olfactory), data=new_df)

# auditory_df = filter(both, DominantModality == "Auditory")
# predict(model, auditory_df)